有以下客服（客服999）与用户（李庆辉）对话记录:

In [1]:
data = '''
对话开始 >>
李庆辉 2020-05-15 12:33:50
	你好，可以退货吗
客服999 2020-05-15 12:33:55 >>
	工号999很高兴为您服务~。
客服999 2020-05-15 12:33:53
	您好
客服999 2020-05-15 12:34:04
	您可以自己操作申请取消订单的。
李庆辉 2020-05-15 12:34:04
	退款多久到账呢？
客服999 2020-05-15 12:34:28
	一般1-7个工作日
李庆辉 2020-05-15 12:35:01
	OMG! 好久呢
李庆辉 2020-05-15 12:40:55
	能不能快点
客服999 2020-05-15 12:42:23
	一般情况下很快就会到账的。
李庆辉 2020-05-15 12:43:04
	OMG! 好久呢
客服999 2020-05-15 12:44:01
	一般情况下很快就会到账的。
对话结束  >>
	长时间未回复，对话结束
'''

In [2]:
import pandas as pd
import numpy as np
from io import StringIO
df = pd.read_csv(StringIO(data), names=['chats'], dtype='string')

In [3]:
pd.show_versions()

D:\anaconda\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(



INSTALLED VERSIONS
------------------
commit           : None
python           : 3.8.3.final.0
python-bits      : 64
OS               : Windows
OS-release       : 7
machine          : AMD64
processor        : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : Chinese (Simplified)_People's Republic of China.936

pandas           : 1.0.5
numpy            : 1.18.5
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.1.1
setuptools       : 49.2.0.post20200714
Cython           : 0.29.21
pytest           : 5.4.3
hypothesis       : None
sphinx           : 3.1.2
blosc            : None
feather          : None
xlsxwriter       : 1.2.9
lxml.etree       : 4.5.2
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.16.1
pandas_datareader: None
bs4              : 4.9.1
bottleneck       : 1.3.2
fastparquet      : None
gcsfs 

## 首次响应时长

In [4]:
(df.loc[~(df.chats.str.endswith('>>')) & ~(df.chats.str.contains('\t'))] # 排除系统内容
 .assign(name=lambda x: x.chats.str.split().str[0]) # 提取双方昵称
 # 判断是否客服（客服昵称 的客服字样）
 .assign(staff=lambda x: x.name.str.contains('客服'))
 .assign(time=lambda x: pd.to_datetime(x.chats.str[-20:])) # 提取并转换为时间类型
 .sort_values('time') # 按先后时间排期
 .assign(first=lambda x: x[x.staff==True].time.min() - x[x.staff==False].time.min()) # 首次响应时长
)

,chats,name,staff,time,first
1,李庆辉 2020-05-15 12:33:50,李庆辉,False,2020-05-15 12:33:50,00:00:03
5,客服999 2020-05-15 12:33:53,客服999,True,2020-05-15 12:33:53,00:00:03
7,客服999 2020-05-15 12:34:04,客服999,True,2020-05-15 12:34:04,00:00:03
9,李庆辉 2020-05-15 12:34:04,李庆辉,False,2020-05-15 12:34:04,00:00:03
11,客服999 2020-05-15 12:34:28,客服999,True,2020-05-15 12:34:28,00:00:03
13,李庆辉 2020-05-15 12:35:01,李庆辉,False,2020-05-15 12:35:01,00:00:03
15,李庆辉 2020-05-15 12:40:55,李庆辉,False,2020-05-15 12:40:55,00:00:03
17,客服999 2020-05-15 12:42:23,客服999,True,2020-05-15 12:42:23,00:00:03
19,李庆辉 2020-05-15 12:43:04,李庆辉,False,2020-05-15 12:43:04,00:00:03
21,客服999 2020-05-15 12:44:01,客服999,True,2020-05-15 12:44:01,00:00:03


## 平均响应时长

In [5]:
(df.loc[~(df.chats.str.endswith('>>')) & ~(df.chats.str.contains('\t'))] # 排除系统内容
 .assign(name=lambda x: x.chats.str.split().str[0]) # 提取双方昵称
 .assign(staff=lambda x: x.name.str.contains('客服')) # 判断是否客服（客服昵称 的客服字样）
 .assign(time=lambda x: pd.to_datetime(x.chats.str[-20:])) # 提取并转换为时间类型
 .sort_values('time') # 按先后时间排期
 # 开始求平均时长，保留一方连续发言情况中的第一条
 .loc[lambda x: (x[['name']].shift() != x[['name']]).any(axis=1)]
 .assign(diff=lambda x: x.time.diff()) # 求先后对话的时差
 .assign(avg=lambda x: x['diff'].mean().seconds) # 所有的时差求平均数，秒
)

,chats,name,staff,time,diff,avg
1,李庆辉 2020-05-15 12:33:50,李庆辉,False,2020-05-15 12:33:50,NaT,87
5,客服999 2020-05-15 12:33:53,客服999,True,2020-05-15 12:33:53,00:00:03,87
9,李庆辉 2020-05-15 12:34:04,李庆辉,False,2020-05-15 12:34:04,00:00:11,87
11,客服999 2020-05-15 12:34:28,客服999,True,2020-05-15 12:34:28,00:00:24,87
13,李庆辉 2020-05-15 12:35:01,李庆辉,False,2020-05-15 12:35:01,00:00:33,87
17,客服999 2020-05-15 12:42:23,客服999,True,2020-05-15 12:42:23,00:07:22,87
19,李庆辉 2020-05-15 12:43:04,李庆辉,False,2020-05-15 12:43:04,00:00:41,87
21,客服999 2020-05-15 12:44:01,客服999,True,2020-05-15 12:44:01,00:00:57,87
